In [12]:
import tdmclient.notebook
await tdmclient.notebook.start()

NodeLockError: Node lock error (current status: busy)

In [4]:
import matplotlib.pyplot as plt
import time
import sys
import cv2
import math

from vision import *
from kalman import *

In [13]:
@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500):
    """
    Sets the motor speeds of the Thymio
    param l_speed: left motor speed
    param r_speed: right motor speed
    """
    global motor_left_target, motor_right_target

    motor_left_target = l_speed
    motor_right_target = r_speed

In [14]:
@tdmclient.notebook.sync_var
def read_motors():
    """
    Read the motor speeds of the Thymio
    """
    global motor_left_speed, motor_right_speed

    return [motor_left_speed, motor_right_speed]

In [14]:
vid = cv2.VideoCapture(0)

In [9]:
# Initialize pygame
pg.init()

h = 600
w = 800
scr = pg.display.set_mode((w, h))

thymio = pg.image.load("thymio.png")
thymio = pg.transform.scale(thymio, (50, 50))

# Set simulation parameters
Ts = 0.01
NUM_STEPS = 300
MEAS_EVERY_STEPS = 10

# main loop
step = 0
running = True
simulate = False
while running:
    for event in pg.event.get():
        if event.type == pg.QUIT:
            pg.quit()
            sys.exit()
        elif event.type == pg.KEYDOWN:
            if event.key == pg.K_RETURN :
                simulate = not simulate
    # Set up environment
    scr.fill((0, 0, 0))
    pg.draw.line(scr, (255, 255, 255), (0, int(h/2)), (w, int(h/2)))
    pg.draw.line(scr, (255, 255, 255), (int(w/2), 0), (int(w/2), h))

    # Start thymio movement
    if simulate:
        step+=1

        # Save expected values and covariance matrices
        mus_pos.append(kf_pos.mean)
        covs_pos.append(kf_pos.cov)
        mus_att.append(kf_att.mean)
        covs_att.append(kf_att.cov)

        # Read sensor data
        video_odometry = odoFetch(vid)
        if not video_odometry:
            pos = video_odometry[1:2]
            angle = video_odometry[3]
        motor_speeds = read_motors()
        speed = (sum(motor_speeds)/2)*0.0005 # magnitude of velocity [m/s]
        angular_speed = (motor_speeds[1]-motor_speeds[0])*0.0005/0.094 # magnitude of angular velocity [rad/s]
        velocity = [speed*np.cos(mus_att[-1][0]), speed*np.sin(mus_att[-1][0])]

        # Estimate pose
        kf_pos.predict(dt=Ts)
        kf_pos.update(meas_value=velocity,
                meas_variance=meas_variance_vel,
                C=np.array([[0, 0, 1, 0], [0, 0, 0, 1]]))
        if not video_odometry:
            kf_pos.update(meas_value=pos,
                meas_variance=meas_variance_pos,
                C=np.array([[1, 0, 0, 0], [0, 1, 0, 0]]))

        # Estimate orientation
        kf_att.predict(dt=Ts)
        kf_att.update(meas_value=angular_speed,
                meas_variance=meas_variance_omega,
                C=np.array([0, 1]))
        if not video_odometry:
            kf_pos.update(meas_value=angle,
                meas_variance=meas_variance_att,
                C=np.array([1, 0]))

        # Update thymio position
        move(kf_pos.mean[0], kf_pos.mean[1], kf_att.mean[0], w, h, thymio, scr)
        drawLine([mu[0] for mu in mus_pos], [mu[1] for mu in mus_pos], w, h, scr)

        # Flow control
        if step > 100:
            motors(-200, 200)
    else:
        move(kf_pos.mean[0], kf_pos.mean[1], kf_att.mean[0], w, h, thymio, scr)
        motors(0, 0)

    # Update screen
    pg.display.flip()
    time.sleep(Ts)

motors(0, 0)

NameError: name 'motors' is not defined

In [ ]:
stop()